# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [3]:
import pandas as pd
import numpy as np
import re

url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)
df.sample(5)


,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
1635,1635,DA10870,Arizona,6289.799089,No,Extended,Bachelor,1/8/11,Employed,M,...,2.0,5,Personal Auto,Personal L2,Offer1,Call Center,563.727058,Four-Door Car,Small,NaN
10099,10099,DH51686,Arizona,4048.722426,No,Premium,Bachelor,2/20/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L2,Offer1,Web,566.400000,Four-Door Car,Small,NaN
6702,6702,YY46839,Washington,8482.456771,No,Premium,High School or Below,1/30/11,Employed,F,...,0.0,9,Corporate Auto,Corporate L2,Offer1,Agent,763.200000,Two-Door Car,Medsize,NaN
4904,4904,PY50779,Arizona,7060.212867,No,Extended,High School or Below,1/27/11,Employed,M,...,0.0,3,Personal Auto,Personal L1,Offer1,Call Center,134.684555,NaN,NaN,A
8731,8731,AA10041,California,7901.744238,No,Basic,High School or Below,2/2/11,Unemployed,M,...,0.0,5,Personal Auto,Personal L3,Offer1,Call Center,1258.327804,SUV,Medsize,A


In [4]:
df.shape

(10910, 26)

In [6]:
df.rename(columns={'Response': 'response', 'Total Claim Amount' : 'tcm'}, inplace=True)

In [7]:
df.dtypes

Unnamed: 0                         int64
Customer                          object
State                             object
Customer Lifetime Value          float64
response                          object
Coverage                          object
Education                         object
Effective To Date                 object
EmploymentStatus                  object
Gender                            object
Income                             int64
Location Code                     object
Marital Status                    object
Monthly Premium Auto               int64
Months Since Last Claim          float64
Months Since Policy Inception      int64
Number of Open Complaints        float64
Number of Policies                 int64
Policy Type                       object
Policy                            object
Renew Offer Type                  object
Sales Channel                     object
tcm                              float64
Vehicle Class                     object
Vehicle Size    

In [10]:
df[["Customer Lifetime Value", "Number of Open Complaints","tcm"]] = df[["Customer Lifetime Value", "Number of Open Complaints","tcm"]].round(2)



In [13]:
df1 = df[(df["tcm"] > 1000) & (df["response"] == 'Yes')].copy()
df1.shape

(67, 26)

In [14]:
df2 = df[(df["response"] == 'Yes')].copy()
df2.shape


(1466, 26)

In [19]:
ptg_tcm = df.groupby(['Policy Type', 'Gender'])['tcm'].mean().reset_index()
ptg_tcm



,Policy Type,Gender,tcm
0,Corporate Auto,F,397.799383
1,Corporate Auto,M,462.223437
2,Personal Auto,F,413.239754
3,Personal Auto,M,459.919530
4,Special Auto,F,458.139664
5,Special Auto,M,420.355172


#For both Corporate Auto and Personal Auto policies, males tend to have higher average total claim amounts compared to females.

For Special Auto policies, females have a higher average total claim amount compared to males, which is an exception to the trend seen in the other two policy types.

The highest average total claim amount overall is for males with Personal Auto policies , while the lowest is for females with Corporate Auto policies

In [20]:
state_counts = df['State'].value_counts().reset_index()
state_counts.columns = ['state', 'customer_count']

states_over_500 = state_counts[state_counts['customer_count'] > 500]

# Sort the results in descending order of customer count
states_over_500 = states_over_500.sort_values('customer_count', ascending=False)

# Display the results
states_over_500

,state,customer_count
0,California,3552
1,Oregon,2909
2,Arizona,1937
3,Nevada,993
4,Washington,888


In [21]:
result = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median'])
result

max      min    median
Education            Gender                             
Bachelor             F       73225.96  1904.00  5640.505
                     M       67907.27  1898.01  5548.030
College              F       61850.19  1898.68  5623.610
                     M       61134.68  1918.12  6005.850
Doctor               F       44856.11  2395.57  5332.460
                     M       32677.34  2267.60  5577.670
High School or Below F       55277.45  2144.92  6039.555
                     M       83325.38  1940.98  6286.735
Master               F       51016.07  2417.78  5729.860
                     M       50568.26  2272.31  5579.100